## Exercise 1.4 Hotdog -- no hotdog
This is the poster hand-in project for the course. Please see the associated PDF for instructions.

In [1]:
!pip install torchinfo

In [2]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.models as models
from torchinfo import summary

from typing import Dict, List, Tuple

We always check that we are running on a GPU

In [3]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator


In [4]:
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='/content/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')

    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]

        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y

## Transfer Learning: EfficientNet

### Load Data

In [5]:
# Define helper functions
def plot_pictures(loader: DataLoader, quantity: int):
  images, labels = next(iter(loader))
  plt.figure(figsize=(20,10))

  for i in range(quantity):
      plt.subplot(5,7,i+1)
      plt.imshow(np.swapaxes(np.swapaxes(images[i].numpy(), 0, 2), 0, 1))
      plt.title(['hotdog', 'not hotdog'][labels[i].item()])
      plt.axis('off')

Perform data transform to dataset as in EfficientNetV1 model:
- Resize (128, 128)
- To Tensor (0,1)
- Mean and std normalization


In [6]:
# Define paramenter for tranformer
size = 128
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define transformer
train_transformer = transforms.Compose([transforms.Resize((size, size)),
                                        transforms.RandomRotation(10),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=mean,
                                                             std=std)])
test_transformer = transforms.Compose([transforms.Resize((size, size)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean,
                                                            std=std)])
# Create train and test dataloaders
batch_size = 64
num_workers = os.cpu_count()  # detects number of CPUs
print(f'Number of cores available: {num_workers}')
trainset = Hotdog_NotHotdog(train=True, transform=train_transformer)# Train
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=min(2, num_workers))
testset = Hotdog_NotHotdog(train=False, transform=test_transformer)# Test
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=min(2, num_workers))
# Verify data size
print(f'Size of train dataset: {len(trainset)}, size of test dataset: {len(test_loader)}')

Number of cores available: 2
Size of train dataset: 2047, size of test dataset: 30


### Set up pretrained model

In [7]:
# Set up weights
weights = models.EfficientNet_B7_Weights.DEFAULT # .DEFAULT = best available weights for ImageNet
model = models.efficientnet_b7(weights=weights).to(device)

Efficientnet_b7 comes in three main parts:

- features - A collection of convolutional layers and other various activation layers to learn a base representation of vision data (this base representation/collection of layers is often referred to as features or feature extractor, "the base layers of the model learn the different features of images").
- avgpool - Takes the average of the output of the features layer(s) and turns it into a feature vector.
- classifier - Turns the feature vector into a vector with the same dimensionality as the number of required output classes (since efficientnet_b0 is pretrained on ImageNet and because ImageNet has 1000 classes, out_features=1000 is the default).

In [8]:
# Get a summary of EfficientNet model
summary(model=model,
        input_size=(32, 3, size, size),
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 128, 128]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 128, 128]    [32, 2560, 4, 4]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 128, 128]    [32, 64, 64, 64]     --                   True
│    │    └─Conv2d (0)                                       [32, 3, 128, 128]    [32, 64, 64, 64]     1,728                True
│    │    └─BatchNorm2d (1)                                  [32, 64, 64, 64]     [32, 64, 64, 64]     128                  True
│    │    └─SiLU (2)                                         [32, 64, 64, 64]     [32, 64, 64, 64]     --                   --
│    └─Sequential (1)                                        [32, 64, 64, 64]     [32, 32, 64,

Now, we are going to freeze some base layers of the pretrained model and adjust the output layers (Classifier layer)

In [9]:
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.features.parameters():
    param.requires_grad = False

In [10]:
# Adapt the classifier layers
torch.manual_seed(42)
torch.cuda.manual_seed(42)

output_shape = 2 # HotDog and NotHotDog
num_features = (model.classifier[-1].in_features)

print(f"Adapting model...")
print(f"Num. of classes: {output_shape} ")
print(f"Num. of features: {num_features}")

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=num_features,
                    out_features=output_shape,
                    bias=True)).to(device)


Adapting model...
Num. of classes: 2 
Num. of features: 2560


In [11]:
# Print model summary of the adapted model
# # Do a summary *after* freezing the features and changing the output classifier layer (uncomment for actual output)
summary(model,
        input_size=(32, 3, size, size),
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 128, 128]    [32, 2]              --                   Partial
├─Sequential (features)                                      [32, 3, 128, 128]    [32, 2560, 4, 4]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 128, 128]    [32, 64, 64, 64]     --                   False
│    │    └─Conv2d (0)                                       [32, 3, 128, 128]    [32, 64, 64, 64]     (1,728)              False
│    │    └─BatchNorm2d (1)                                  [32, 64, 64, 64]     [32, 64, 64, 64]     (128)                False
│    │    └─SiLU (2)                                         [32, 64, 64, 64]     [32, 64, 64, 64]     --                   --
│    └─Sequential (1)                                        [32, 64, 64, 64]     [32, 

### Training model

In [12]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train_model(model: nn.Module,
                train_dataloader: DataLoader,
                test_dataloader: DataLoader,
                optimizer: torch.optim.Optimizer,
                loss_fn: nn.Module,
                epochs: int,
                device: torch.device)->Dict[str, List]:
                    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results





In [13]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Train model
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
results = train_model(model=model,
                       train_dataloader=train_loader,
                       test_dataloader=test_loader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=5,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5632 | train_acc: 0.7421 | test_loss: 0.5236 | test_acc: 0.7573
Epoch: 2 | train_loss: 0.4420 | train_acc: 0.8158 | test_loss: 0.4837 | test_acc: 0.7859


KeyboardInterrupt: 

### Evaluation

In [ ]:
# Get the plotWeight the 2D activations by the average gradient_loss_curves() function from helper_functions.py, download the file if we don't have it
try:
    from helper_functions import plot_loss_curves
except:
    print("[INFO] Couldn't find helper_functions.py, downloading...")
    with open("helper_functions.py", "wb") as f:
        import requests
        request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
        f.write(request.content)
    from helper_functions import plot_loss_curves

# Plot the loss curves of our model
plot_loss_curves(results)

### Prediction

In [ ]:
# 1. Take in a trained model, class names, image path, image size, a transform and target device
def pred_and_plot_image(model: torch.nn.Module,
                        image_path: List[str],
                        class_names: List[str],
                        image_size: Tuple[int, int] = (128, 128),
                        transform: transforms = None,
                        device: torch.device=device):

    """
    Predict and plot images in a 4x3 grid showing predicted class and probability.
    """

    # Ensure model is on correct device
    model.to(device)
    model.eval()

    # Create transform if not provided
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean,
                                 std=std),
        ])

    # Create figure
    fig, axes = plt.subplots(4, 3, figsize=(12, 16))
    axes = axes.flatten()

    # Loop through up to 12 images
    for i, image_path in enumerate(image_path[:len(image_path)]):
        img = Image.open(image_path).convert("RGB")
        transformed_image = image_transform(img).unsqueeze(0).to(device)

        with torch.inference_mode():
            pred = model(transformed_image)

        pred_probs = torch.softmax(pred, dim=1)
        pred_label = torch.argmax(pred_probs, dim=1).item()
        pred_prob = pred_probs[0, pred_label].item()

        # Plot image
        axes[i].imshow(img.resize(image_size))
        axes[i].set_title(f"{class_names[pred_label]} ({pred_prob*100:.1f}%)", fontsize=10)
        axes[i].axis("off")

    # Turn off any empty subplots
    for j in range(len(image_path), 12):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

**Saliency map**

In [ ]:


def add_gaussian_noise(tensor: torch.Tensor, std: float = 0.1) -> torch.Tensor:
    """Adds Gaussian noise to a tensor image."""
    noise = torch.randn_like(tensor) * std
    return torch.clamp(tensor + noise, 0.0, 1.0)

def pred_sal(model: torch.nn.Module,
             image_path: str,
             class_names: List[str],
             image_size: Tuple[int, int] = (128, 128),
             transform: transforms.Compose = None,
             device: torch.device = torch.device("cpu"),
             n_samples: int = 10,
             noise_std: float = 0.25):
    """
    Predicts on a single image and visualizes the original image
    with its SmoothGrad saliency map.
    """

    # --- Load image ---
    img = Image.open(image_path).convert("RGB")

    # --- Define transform if not provided ---
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    # --- Apply transform and prepare tensor ---
    X = transform(img).unsqueeze(0).to(device)
    X.requires_grad_()

    # --- Model prediction ---
    model.to(device)
    model.eval()
    scores = model(X)
    probs = torch.softmax(scores, dim=1)
    pred_label = torch.argmax(probs, dim=1)

    # --- SmoothGrad: accumulate gradients over noisy samples ---
    saliency_total = torch.zeros_like(X)
    for _ in range(n_samples):
        noisy_X = X + torch.randn_like(X) * noise_std
        noisy_X.requires_grad_()
        noisy_X.retain_grad()
        model.zero_grad()
        out = model(noisy_X)
        score_max = out[0, pred_label[0]]
        score_max.backward()
        saliency_total += noisy_X.grad.abs()

    saliency = saliency_total / n_samples
    saliency = saliency.max(dim=1)[0].detach().cpu()  # collapse channels

    # --- Normalize for plotting ---
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-10)

    # --- Plot original image and saliency map side by side ---
    plt.figure(figsize=(10,5))

    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Pred: {class_names[pred_label[0]]}\nProb: {probs.max():.3f}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(saliency[0], cmap='hot')
    plt.title("SmoothGrad Saliency")
    plt.axis('off')

    plt.show()




In [ ]:
# Get a random list of image paths from test set
import random
from pathlib import Path

num_images_to_plot = 10
test_image_path_list = list(Path('/content/hotdog_nothotdog/test').glob("*/*.jpg")) # get list all image paths from test data
test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                       k=num_images_to_plot) # randomly select 'k' image paths to pred and plot
class_names = ['HotDog', 'NotHogDog']
# Make predictions on and plot the images
for image_path in test_image_path_sample:
    pred_sal(model=model,
              image_path=image_path,
                        class_names=class_names,
                        # transform=weights.transforms(), # optionally pass in a specified transform from our pretrained model weights
                        image_size=(128, 128))

In [ ]:
def saliency_map(model: torch.nn.Module,
                 dataloader: torch.utils.data.DataLoader,
                 device: torch.device = torch.device("cpu")) -> None:
    """Visualizes a saliency map for the first image in a DataLoader batch."""

    model.eval()  # Make sure model is in eval mode

    # Get first batch
    X, y = next(iter(dataloader))
    X, y = X.to(device), y.to(device)
    X.requires_grad_()  # Track gradients w.r.t input

    # Forward pass
    outputs = model(X)
    predicted_class = outputs.argmax(dim=1)

    # Compute loss for the predicted class of the first image
    loss = outputs[0, predicted_class[0]]  # Only first image
    model.zero_grad()
    loss.backward()

    # Saliency = max absolute gradient across channels
    saliency = X.grad.data.abs().max(dim=1)[0].cpu()

    # Plot the first image's saliency
    import matplotlib.pyplot as plt
    plt.imshow(saliency[0], cmap=plt.cm.hot)
    plt.axis('off')
    plt.title(f"Predicted class: {predicted_class[0].item()}")
    plt.show()



saliency_map(model=model, dataloader=train_loader, device=device)

In [ ]:


def add_gaussian_noise(tensor: torch.Tensor, std: float = 0.1) -> torch.Tensor:
    """Adds Gaussian noise to a tensor image."""
    noise = torch.randn_like(tensor) * std
    noisy_tensor = tensor + noise
    return torch.clamp(noisy_tensor, 0.0, 1.0)

def pred_sal(model: torch.nn.Module,
             image_path: str,
             class_names: List[str],
             image_size: Tuple[int, int] = (128, 128),
             transform: transforms.Compose = None,
             device: torch.device = torch.device("cpu"),
             n_samples: int = 10,
             noise_std: float = 0.1):
    """
    Predicts on a single image and visualizes the original image
    with its SmoothGrad saliency map side by side.
    """

    # --- Load and transform image ---
    img = Image.open(image_path).convert("RGB")
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
    X = transform(img).unsqueeze(0).to(device)
    X.requires_grad_()

    # --- Prediction ---
    model.to(device)
    model.eval()
    outputs = model(X)
    probs = torch.softmax(outputs, dim=1)
    pred_label = torch.argmax(probs, dim=1)

    # --- Compute SmoothGrad saliency ---
    saliency_total = torch.zeros_like(X)
    for _ in range(n_samples):
        noisy_X = add_gaussian_noise(X, std=noise_std)
        noisy_X.requires_grad_()
        model.zero_grad()
        out = model(noisy_X)
        loss = out[0, pred_label[0]]
        loss.backward()
        saliency_total += noisy_X.grad.data.abs()

    saliency = saliency_total / n_samples
    saliency = saliency.max(dim=1)[0].cpu()  # collapse channels

    # --- Plot original image and SmoothGrad saliency map side by side ---
    plt.figure(figsize=(10, 5))

    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Pred: {class_names[pred_label[0]]}\nProb: {probs.max():.3f}")
    plt.axis('off')

    # SmoothGrad saliency map
    plt.subplot(1, 2, 2)
    plt.imshow(saliency[0], cmap=plt.cm.hot)
    plt.title("SmoothGrad Saliency")
    plt.axis('off')

    plt.show()


In [ ]:
# Get a random list of image paths from test set
import random
from pathlib import Path

num_images_to_plot = 12
test_image_path_list = list(Path('/content/hotdog_nothotdog/test').glob("*/*.jpg")) # get list all image paths from test data
test_image_path_sample = random.sample(population=test_image_path_list, # go through all of the test image paths
                                       k=num_images_to_plot) # randomly select 'k' image paths to pred and plot
class_names = ['Hot_Dog', 'Not_Hot_Dog']
# Make predictions on and plot the images
for image_path in test_image_path_sample:
    pred_and_plot_image(model=model,
                        image_path=image_path,
                        class_names=class_names,
                        image_size=(128, 128))